In [2]:
import sys 
sys.path.append("../../../llm_engineering")
from api_clients import create_clients
import gradio as gr 

In [3]:
clients = create_clients()

In [4]:
system_prompt = """
  You are a helpful assistant that generates proper comments for code provided by the user.
  The comments should be properly structured but shouldn't be very detail.
  Yet they should be simple enought to convery what the code is doing, its inputs, outputs.
  The code can be in any lanugagues, but you should comment in its proper format.
  Feel free to add additional comments on code snippets where there's significant complexity, 
  for example in Mathematical formulas. 
  You should NOT change any of the code provided apart from adding comments.
  Event if the code has bugs/flaws, DO NOT CHANGE it.
"""

In [5]:
def generate_model_response(code):
    if not code.strip():
        return "⚠️ Please enter some code first."
    
    try:
        user_prompt = f"""Please add helpful comments to explain this code. Preserve all original code exactly.
        Keep the comments concise but informative. Focus on explaining:
        - The purpose of functions/classes
        - Complex logic or algorithms
        - Important parameters and return values
        - Any non-obvious implementation details

        Here's the code to comment:

        ```
        {code}
        ```
        """
        
        response = clients["groq"].chat.completions.create(
            model=clients["models"]["GROQ_MODEL"],
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            stream=True
        )
        
        full_response = ""
        with response as stream:
            for chunk in stream:
                if hasattr(chunk.choices[0].delta, "content"):
                    content = chunk.choices[0].delta.content
                    if content:
                        full_response += content
                        # For streaming updates in the UI
                        yield full_response.replace("```", "").strip()
        
        # Final cleanup and return
        return full_response.replace("```", "").strip()
    
    except Exception as e:
        return f"⚠️ Error: {str(e)}"

In [6]:

with gr.Blocks(theme=gr.themes.Citrus()) as ui:
    gr.Markdown("""
    # 🤖 Code Commenter
    Upload your code or use the templates to get started. Select a language and click 'Add Comments' to generate helpful comments.
    """)
    
    with gr.Row():
        with gr.Column(scale=1):
            language = gr.Dropdown(
                choices=["Python", "Java", "C++", "Scala"],
                value="Python",
                label="Programming Language",
                info="Select the language of your code"
            )
      
            button = gr.Button("Add Comments 🚀", variant="primary")
            
        with gr.Column(scale=2):
            original_code = gr.Code(
                label="Original Code",
                language="python",
                lines=12,
                value=""  # Initialize empty
            )
            commented_code = gr.Code(
                label="Commented Code",
                language="python",
                lines=12,
                interactive=False,
                value=""  # Initialize empty
            )
    
    # Update code language when language dropdown changes
    def update_language(lang):
        lang = lang.lower()
        # Handle C++ special case
        if lang == "c++":
            lang = "cpp"
        return gr.update(language=lang), gr.update(language=lang)
    
    # Handle language change
    language.change(
        fn=update_language,
        inputs=[language]
    )
    
    # Stream the comments as they're generated
    button.click(
        fn=generate_model_response,
        inputs=[original_code],
        outputs=commented_code,
        show_progress="full"
    )

ui.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
